# Data Modeling Concepts

![Image](./images/data_modeling.JPG)

---

## Logical vs. Physical Model

- __Logical Model:__ After obtaining the business requirements, the logical model describes the data to a great extent, but it does not take part in implementing the database.

- __Physical Model:__ A physical data model is a model that helps to implement the database.


![Image](./images/logical_physical.JPG)

---

## Multi-Hop Architecture

Multi-hop architecture uses tables that correspond to different quality levels in the data engineering pipeline, progressively adding structure to the data. One example of this approach is the __Medallion Architecture__ typically used in the Lakehouse data architecture.

![Image](./images/medallion.JPG)

__Bronze layer (raw data):__ The Bronze layer is where we land all the data from external source systems. The table structures in this layer correspond to the source system table structures "as-is," along with any additional metadata columns that capture the load date/time, process ID, etc. The focus in this layer is quick Change Data Capture and the ability to provide an historical archive of source (cold storage), data lineage, auditability, reprocessing if needed without rereading the data from the source system.


__Silver layer (cleansed and conformed data):__ In the Silver layer the data from the Bronze layer is matched, merged, conformed and cleansed ("just-enough") so that the Silver layer can provide an "Enterprise view" of all its key business entities, concepts and transactions. (e.g. master customers, stores, non-duplicated transactions and cross-reference tables).


__Gold layer (curated business-level tables):__ Data in the Gold layer is typically organized in consumption-ready "project-specific" databases. The Gold layer is for reporting and uses more de-normalized and read-optimized data models with fewer joins. The final layer of data transformations and data quality rules are applied here. Final presentation layer of projects such as Customer Analytics, Product Quality Analytics, Inventory Analytics, Customer Segmentation, Product Recommendations, Marking/Sales Analytics etc., fit in this layer. We see a lot of Kimball style star schema-based data models or Inmon style Data marts fit in this Gold Layer.

---

## Data Warehouse

Data warehouse are a group of relational databases accesible across an organization. There are two main types:

- OLAP (On-Line Analytical Processing)

- OLTP (On-Line Transaction Processing)


![Image](./images/olap_oltp.png)


---

## Star Schema

Is the most traditional business intelligence data modelling system developed by Ralph Kimball in 1996 with a "bottom-up" approach. The star schema consists of one or more __fact tables__ referencing any number of __dimension tables__.

- Dimension tables support filtering and grouping

- Fact tables support aggregation

---

![Image](./images/star_models/Sales.png)


In [ ]:
# imports and config

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Fact table

fact_table = pd.read_csv('./datasets/modeling/Sales_short.csv')
#fact_table = pd.read_csv('../../../data/Sales.csv')
fact_table.info()

In [ ]:
fact_table.head()

In [ ]:
# Memory usage function (MB)

def memory_usage(df_list):
    return f'{sum([df.memory_usage().sum() / 1000000 for df in df_list])} megabytes' 

In [ ]:
memory_usage([fact_table])

In [ ]:
# Dimension table

dimension_table = pd.read_csv('./datasets/modeling/Channel.csv')
dimension_table.columns = ['channelKey', 'ChannelName']
dimension_table.info()

In [ ]:
dimension_table

In [ ]:
memory_usage([dimension_table])

In [ ]:
# Many dimensions

channel_table = pd.read_csv('./datasets/modeling/Channel.csv')
channel_table.columns = ['channelKey', 'ChannelName']
store_table = pd.read_csv('./datasets/modeling/Stores.csv')
product_table = pd.read_csv('./datasets/modeling/Product.csv')
promotion_table = pd.read_csv('./datasets/modeling/Promotion.csv')

In [ ]:
channel_table

In [ ]:
store_table

In [ ]:
product_table

In [ ]:
promotion_table

In [ ]:
normalized_model = memory_usage([fact_table,
                                 channel_table,
                                 store_table,
                                 product_table,
                                 promotion_table])
normalized_model

In [ ]:
%%time

# Model denormalization

denormalized_table = pd.merge(fact_table,channel_table).merge(store_table).merge(product_table).merge(promotion_table)
denormalized_table.info()

In [ ]:
denormalized_model = memory_usage([denormalized_table])
denormalized_model

In [ ]:
ratio = round(float(denormalized_model.split()[0])/float(normalized_model.split()[0]), 1)
f'The model size is multiplied by {ratio} in memory size'

In [ ]:
# Table aggregation (summarization)

fact_table.index = pd.to_datetime(fact_table['DateKey'], format='%Y-%m-%d %H:%M:%S')
fact_table_agg = fact_table.groupby([fact_table.index.month, fact_table.index.year])[['TotalCost']].sum()
#pd.set_option('display.float_format', lambda x: '%.3f' % x)
fact_table_agg

---

## Cardinality

__Many to one (*:1):__ A many-to-one relationship is the most common, default type of relationship. It means the column in a given table can have more than one instance of a value, and the other related table, often know as the lookup table, has only one instance of a value.

__One to one (1:1):__ In a one-to-one relationship, the column in one table has only one instance of a particular value, and the other related table has only one instance of a particular value.

__One to many (1:*):__ In a one-to-many relationship, the column in one table has only one instance of a particular value, and the other related table can have more than one instance of a value.

__Many to many (*:*):__ Many-to-many relationship between tables removes requirements for unique values in tables. It also removes previous workarounds, such as introducing new tables only to establish relationships.

---

### More examples!!!

These examples are developed using DBML (Database Markup Language) in [DB Diagram](https://dbdiagram.io/)

---

#### Instagram-like app

![Image](./images/star_models/Instagram.png)

---

#### Logistics

![Image](./images/star_models/Logistics.png)

---

#### Formula 1

![Image](./images/star_models/F1.png)

---

#### Ecommerce

![Image](./images/star_models/Ecommerce.png)

---

#### ML Workflow

![Image](./images/star_models/MLworkflow.png)

---